In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional, Callable, Union
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem,
    function_tool,
    FunctionTool,
    handoff,
    ModelSettings,
)
load_dotenv()

from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph

import json
import random

In [27]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name,

)


In [28]:

class StockData(BaseModel):
    """Model for stock data"""
    price: float = Field(..., description="Current stock price")
    volume: int = Field(..., description="Trading volume")
    pe_ratio: float = Field(..., description="Price-to-earnings ratio")
    market_cap: str = Field(..., description="Market capitalization")

@function_tool
def get_stock_data(symbol: str) -> StockData:
    """Get stock market data for a given symbol"""
    return StockData(**{"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"})

class NewsItem(BaseModel):
    """Model for news items"""
    title: str = Field(..., description="Title of the news article")
    date: str = Field(..., description="Publication date")
    summary: str = Field(..., description="Summary of the article")

@function_tool
def get_news(query: str) -> List[NewsItem]:
    """Get recent news articles about a company"""
    return [
        NewsItem(**
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Q1 Earnings Report",
            "date": "2024-03-21",
            "summary": "Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.",
        }),
        NewsItem(**
        {
            "title": "Tesla Partners with Panasonic",
            "date": "2024-03-22",
            "summary": "Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.",
        }),
        NewsItem(**
        {
            "title": "Tesla Model S Plaid Review",
            "date": "2024-03-23",
            "summary": "The Tesla Model S Plaid receives rave reviews for its performance and technology features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Energy Division Grows",
            "date": "2024-03-24",
            "summary": "Tesla's energy division sees significant growth in solar and battery storage solutions.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Global Expansion Plans",
            "date": "2024-03-25",
            "summary": "Tesla announces plans to expand its manufacturing footprint in Europe and Asia.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Autopilot Update",
            "date": "2024-03-26",
            "summary": "Tesla releases a new update for its Autopilot system, improving safety and navigation features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Stock Performance",
            "date": "2024-03-27",
            "summary": "Tesla's stock shows strong performance in the market, driven by positive investor sentiment.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Sustainability Initiatives",
            "date": "2024-03-28",
            "summary": "Tesla announces new sustainability initiatives aimed at reducing carbon footprint.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Cybertruck Launch Date Announced",
            "date": "2024-03-29",
            "summary": "Tesla announces the official launch date for the highly anticipated Cybertruck.",
        }),
        NewsItem(**
        {
            "title": "Tesla's AI Day 2024 Highlights",
            "date": "2024-03-30",
            "summary": "Tesla's AI Day showcases advancements in artificial intelligence and robotics.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Supercharger Network Expansion",
            "date": "2024-03-31",
            "summary": "Tesla announces plans to expand its Supercharger network across North America.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Model 3 Production Update",
            "date": "2024-03-17",
            "summary": "Tesla increases production capacity for the Model 3 to meet growing demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        }),
        NewsItem(**
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        }),
    ]

class Refund(BaseModel):
    flight_id: str = Field(..., description="Flight ID")
    amount: float = Field(..., description="Refund amount")



In [29]:
stockdata = await get_stock_data.on_invoke_tool(None, json.dumps({"symbol": "TSLA"}))
print(stockdata)
news = await get_news.on_invoke_tool(None, json.dumps({"query": "Tesla"}))
print(news)


price=180.25 volume=1000000 pe_ratio=65.4 market_cap='700B'
[NewsItem(title='Tesla Expands Cybertruck Production', date='2024-03-20', summary='Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.'), NewsItem(title="Tesla's Q1 Earnings Report", date='2024-03-21', summary='Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.'), NewsItem(title='Tesla Partners with Panasonic', date='2024-03-22', summary='Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.'), NewsItem(title='Tesla Model S Plaid Review', date='2024-03-23', summary='The Tesla Model S Plaid receives rave reviews for its performance and technology features.'), NewsItem(title="Tesla's Energy Division Grows", date='2024-03-24', summary="Tesla's energy division sees significant growth in solar and battery storage solutions."), NewsItem(title="Tesla's Global Expansion Plans", date='202

In [30]:
financial_analyst = Agent(
    name="financial_analyst",
    model=model_name,
    tools=[get_stock_data],
    output_type=StockData,
    instructions="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    ### IMPORTANT ###
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a financial analyst.
    You perform analysis of the stock market data.
    you must handoff back to the planner agent."""
)


news_analyst = Agent(
    name="news_analyst",
    model=model_name,
    tools=[get_news],
    output_type=List[NewsItem],
    instructions="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a news analyst.
    You retrieve news articles to analyze.
    you must handoff back to the planner agent."""
)


writer = Agent(
    name="writer",
    model=model_name,
    tools=[],
    output_type=str,
    instructions="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    The report should be easy to understand for a general audience.
    The report is formatted as a markdown document.
    Always handoff back to planner when writing is complete.""",
    handoff_description="""You are a writer.
    You summary a summary of the analysis results.
    you must handoff back to the planner agent."""
)



try:
    draw_graph(financial_analyst, filename="viz/11_planner_financial_analyst.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [31]:
msg = "what is the TSLA stock, write a report"
input_items : list[TResponseInputItem] = [{"content" : msg, "role": "user"}]

In [32]:
# with trace(workflow_name="financial_analyst", group_id="mdsi") as tr:
#     print(f"Tracing url: {get_trace_url(tr)}")

#     result = await Runner.run(
#         financial_analyst,
#         input=input_items,
#         run_config=RunConfig(
#             model=model_name,
#             model_provider=GITHUB_MODEL_PROVIDER
#         )

#     )

#     print(result)

In [54]:
planner_assistant = Agent(
    name="planner",
    model=model_name,
    handoffs=[financial_analyst,news_analyst, writer],
    instructions="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
    handoff_description="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents
    you are responsible for generating the final output.,
    """,

)


financial_analyst.handoffs = [handoff(planner_assistant)]
news_analyst.handoffs = [handoff(planner_assistant)]
# writer.handoffs = [handoff(planner_assistant)]
writer.handoffs = []

try:
    draw_graph(planner_assistant, filename="viz/11_planner_financial_planner_all_handoffs.gv")
except Exception as e:
    print(f"An error occurred: {e}")

try:
    draw_graph(writer, filename="viz/11_planner_writer.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [61]:
with trace(workflow_name="financial_planner", group_id="financial_planner") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    result = await Runner.run(
        planner_assistant,
        input=input_items,
        run_config=RunConfig(
            model=model_name,
            model_provider=GITHUB_MODEL_PROVIDER,
            model_settings=ModelSettings(
                temperature=0.5,
                max_tokens=4000
            ),
        )

    )

    print(result)

Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_0cf9b3b349a746bd8096f0a387f31ca6
RunResult:
- Last agent: Agent(name="writer", ...)
- Final output (str):
    # TSLA Stock Report
    
    ## Overview of Tesla, Inc. (TSLA)
    
    Tesla, Inc. (NASDAQ: TSLA) is a leading electric vehicle (EV) manufacturer known for its innovation in sustainable energy solutions. As of the latest data, TSLA stock is priced at **$180.25** with a market capitalization of approximately **$700 billion**. The stock has a **P/E ratio of 65.4**, indicating high investor expectations for future growth.
    
    ## Current Stock Performance
    
    - **Current Price**: $180.25
    - **Market Capitalization**: $700 billion
    - **P/E Ratio**: 65.4
    - **Trading Volume**: 1,000,000 shares
    
    The stock has shown strong performance in the market, bolstered by positive investor sentiment and strategic business developments.
    
    ## Recent Developments
    
    ### 1. **Production Expa

In [59]:
display(result.final_output)

StockData(price=180.25, volume=1000000, pe_ratio=65.4, market_cap='700B')

In [60]:
result.raw_responses[-1].usage

Usage(requests=1, input_tokens=475, output_tokens=32, total_tokens=507)

In [ ]:
financial_analyst.to

In [ ]:
from agents.version import __version__

In [ ]:
__version__


'0.0.0'

In [ ]:
import pkg_resources

installed_packages = pkg_resources.working_set
for package in installed_packages:
    print(f"{package.key}=={package.version}")

C:\Users\mutaza\AppData\Local\Temp\ipykernel_36592\3885190956.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


deprecated==1.2.18
gitpython==3.1.44
mako==1.3.9
markdown==3.7
markupsafe==3.0.2
pyjwt==2.10.1
pymeta3==0.5.1
pyyaml==6.0.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.14
aiosignal==1.3.2
alembic==1.15.1
aniso8601==10.0.0
annotated-types==0.7.0
anyio==4.9.0
argcomplete==3.6.1
asgiref==3.8.1
asttokens==3.0.0
async-timeout==5.0.1
attrs==25.3.0
autogen-agentchat==0.4.5
autogen-core==0.4.5
autogen-ext==0.4.5
azure-ai-evaluation==1.3.0
azure-ai-inference==1.0.0b9
azure-ai-ml==1.26.0
azure-ai-projects==1.0.0b7
azure-common==1.1.28
azure-core==1.32.0
azure-core-tracing-opentelemetry==1.0.0b12
azure-identity==1.21.0
azure-mgmt-core==1.5.0
azure-monitor-opentelemetry==1.6.5
azure-monitor-opentelemetry-exporter==1.0.0b35
azure-search-documents==11.5.2
azure-storage-blob==12.19.0
azure-storage-file-datalake==12.19.0
azure-storage-file-share==12.21.0
azureml-mlflow==1.59.0.post1
backoff==2.2.1
bcrypt==4.3.0
blinker==1.9.0
build==1.2.2.post1
cachetools==5.5.2
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.